# Imports
Necessary to prevent error codes.

In [1]:
#Standard
import random
from random import random
import os
import csv
import numpy as np
from numpy import asarray
from numpy import savetxt
from scipy.stats import sem
#from stats import mean, median, mode

import datetime as dt
#from uszipcode import SearchEngine
#from spread import variance, standard_deviation, zipped_z_scores
import re

#APIs
import requests as req
import json
#from config import api_key
#from config import gkey

#Pandas
import pandas as pd

# Uploading the list for analyses and making dictionaries for future use

In [2]:
file="ndtcdonorlist.csv"
df=pd.read_csv(file)
print(df.iloc[0:0])
print(df.shape)

state_ab1 = {
    'AL': 'Alabama',
    'AK': 'Alaska',
    'AZ': 'Arizona',
    'AR': 'Arkansas',
    'CA': 'California',
    'CO': 'Colorado',
    'CT': 'Connecticut',
    'DE': 'Delaware',
    'FL': 'Florida',
    'GA': 'Georgia',
    'HI': 'Hawaii',
    'ID': 'Idaho',
    'IL': 'Illinois',
    'IN': 'Indiana',
    'IA': 'Iowa',
    'KS': 'Kansas',
    'KY': 'Kentucky',
    'LA': 'Louisiana',
    'ME': 'Maine',
    'MD': 'Maryland',
    'MA': 'Massachusetts',
    'MI': 'Michigan',
    'MN': 'Minnesota',
    'MS': 'Mississippi',
    'MO': 'Missouri',
    'MT': 'Montana',
    'NE': 'Nebraska',
    'NV': 'Nevada',
    'NH': 'New Hampshire',
    'NJ': 'New Jersey',
    'NM': 'New Mexico',
    'NY': 'New York',
    'NC': 'North Carolina',
    'ND': 'North Dakota',
    'OH': 'Ohio',
    'OK': 'Oklahoma',
    'OR': 'Oregon',
    'PA': 'Pennsylvania',
    'RI': 'Rhode Island',
    'SC': 'South Carolina',
    'SD': 'South Dakota',
    'TN': 'Tennessee',
    'TX': 'Texas',
    'UT': 'Utah',
    'VT': 'Vermont',
    'VA': 'Virginia',
    'WA': 'Washington',
    'WV': 'West Virginia',
    'WI': 'Wisconsin',
    'WY': 'Wyoming'}

state_ab = {
    'Alabama': 'AL',
    'Alaska': 'AK',
    'American Samoa': 'AS',
    'Arizona': 'AZ',
    'Arkansas': 'AR',
    'California': 'CA',
    'Colorado': 'CO',
    'Connecticut': 'CT',
    'Delaware': 'DE',
    'District of Columbia': 'DC',
    'Florida': 'FL',
    'Georgia': 'GA',
    'Guam': 'GU',
    'Hawaii': 'HI',
    'Idaho': 'ID',
    'Illinois': 'IL',
    'Indiana': 'IN',
    'Iowa': 'IA',
    'Kansas': 'KS',
    'Kentucky': 'KY',
    'Louisiana': 'LA',
    'Maine': 'ME',
    'Maryland': 'MD',
    'Massachusetts': 'MA',
    'Michigan': 'MI',
    'Minnesota': 'MN',
    'Mississippi': 'MS',
    'Missouri': 'MO',
    'Montana': 'MT',
    'Nebraska': 'NE',
    'Nevada': 'NV',
    'New Hampshire': 'NH',
    'New Jersey': 'NJ',
    'New Mexico': 'NM',
    'New York': 'NY',
    'North Carolina': 'NC',
    'North Dakota': 'ND',
    'Northern Mariana Islands':'MP',
    'Ohio': 'OH',
    'Oklahoma': 'OK',
    'Oregon': 'OR',
    'Pennsylvania': 'PA',
    'Puerto Rico': 'PR',
    'Rhode Island': 'RI',
    'South Carolina': 'SC',
    'South Dakota': 'SD',
    'Tennessee': 'TN',
    'Texas': 'TX',
    'Utah': 'UT',
    'Vermont': 'VT',
    'Virgin Islands': 'VI',
    'Virginia': 'VA',
    'Washington': 'WA',
    'West Virginia': 'WV',
    'Wisconsin': 'WI',
    'Wyoming': 'WY'}

df = df.drop(['Last Name', 'Address 1', 'Address 2', 'City','Zip', 'Employer', 'Occupation','Home Phone', 'Work Phone', 'Mobile Phone'], axis = 1)
df = df.dropna()

C:\Users\TEMP\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3146: DtypeWarning: Columns (3) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


Empty DataFrame
Columns: [First Name, Last Name, Address 1, Address 2, City, State, Zip, Employer, Occupation, Email, Home Phone, Work Phone, Mobile Phone]
Index: []
(199815, 13)


# Now for the fun to start
Asking the user for their inputs

In [3]:
#inputs
name  = input("What is your name? ")
name = name.lower()
name = name.capitalize()

email = input("What is your email address? ")
email = email.split('@')
email = email[1]
state1=0
state2=0

state = input("What state do you live in? ")

if len(state)==2:
    state1=state_ab1[state]
    state2=state

if len(state)>2:
    state1=state_ab2[state]
    state2=state
    
print("Just so we're on the same page, your name is",name,", your email address belongs in the",email,"domain, and you live in",state1,"/",state2,".")

What is your name? John
What is your email address? john@gmail.com
What state do you live in? PA
Just so we're on the same page, your name is John , your email address belongs in the gmail.com domain, and you live in Pennsylvania / PA .


# Fixing the inputs to fit the data

In [4]:
#name
df['First Name']=df['First Name'].str.lower()
df['First Name']=df['First Name'].str.capitalize()
df = df.rename(columns={"First Name": "name"})

#email
df['email1'] = df['Email'].str.split('@').str[1]
df = df.drop(['Email'],axis=1)
df['email1'] = df['email1'].str.lower()
df = df.rename(columns={"email1": "email"})

#state

if len(state1)>len(state2):
    state2 = state
else:
    state1 = state

# Calculating the statistics around the users name, email domain, and state

In [5]:
#Statistics
#Name stats

name_count = len(df[df['name'] == name])
name_perc  = name_count/len(df['name'])
name_perc = round(name_perc,3)

#Email stats
email_count = len(df[df['email'] == email])
email_perc  = email_count/len(df['email'])
email_perc = round(email_perc,3)

#State stats
state_count = len(df[df['State'] == state])
state_perc  = state_count/len(df['State'])
state_perc = round(state_perc,3)

#General
state_stats = df[df['State'] == state]
name_stats = df[df['name'] == name]
email_stats = df[df['email'] == email]

state_n = state_stats[state_stats['name'] == name]
state_e = state_stats[state_stats['email'] == email]
name_e= name_stats[name_stats['email'] == email]
name_s = name_stats[name_stats['State'] == state]
email_s = email_stats[email_stats['State'] == state]

state_name = len(state_n)
sn_perc = state_name/state_count
sn_perc = round(sn_perc,2)

state_email = len(state_e)
se_perc = state_email/state_count
se_perc = round(se_perc,2)

name_email = len(name_e)
ne_perc = name_email/name_count
ne_perc = round(ne_perc,2)

name_state = len(name_s)
ns_perc = name_state/name_count
ns_perc = round(ns_perc,2)

email_state = len(email_s)
es_perc = email_state/email_count
es_perc = round(es_perc,2)

print("In this list of donors, your name (",name,") appears",name_count,"times, and acconuts for ~",name_perc * 100,"% of the names in the whole list.")
print("...")
print("In this list of donors, your email domain (",email,") appears",email_count,"times, and acconuts for ~",email_perc * 100,"% of the email domains in the whole list.")
print("...")
print("In this list of donors, your home-state (",state,") appears",state_count,"times, and acconuts for ~",state_perc * 100,"% of the home states in the whole list.")
print("...")
print("Of the donors from your home state (",state,"),",state_name,"share your name, accounting for ~",sn_perc,"% of the donors from your home state.")
print("Of the donors from your home state (",state,"),",state_email,"share your email domain, accounting for ~",se_perc,"% of the donors from your home state.")
print("...")
print("Of the donors sharing your name (",name,"),",name_email,"share your email domain, accounting for ~",ne_perc,"% of the donors sharing your name.")
print("Of the donors sharing your name (",name,"),",name_state,"share your home state, accounting for ~",ns_perc,"% of the donors sharing your name.")

In this list of donors, your name ( John ) appears 2752 times, and acconuts for ~ 1.4000000000000001 % of the names in the whole list.
...
In this list of donors, your email domain ( gmail.com ) appears 72888 times, and acconuts for ~ 36.5 % of the email domains in the whole list.
...
In this list of donors, your home-state ( PA ) appears 7660 times, and acconuts for ~ 3.8 % of the home states in the whole list.
...
Of the donors from your home state ( PA ), 135 share your name, accounting for ~ 0.02 % of the donors from your home state.
Of the donors from your home state ( PA ), 2653 share your email domain, accounting for ~ 0.35 % of the donors from your home state.
...
Of the donors sharing your name ( John ), 907 share your email domain, accounting for ~ 0.33 % of the donors sharing your name.
Of the donors sharing your name ( John ), 135 share your home state, accounting for ~ 0.05 % of the donors sharing your name.
